In [1]:
#keras/blob/master/examples/cifar10_cnn

In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import numpy as np

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)


In [3]:
batch_size = 32       #can increase this until memory error. also watch nvidia-smi to check memory use.
num_classes = 10      #can check this from data once loaded.
epochs = 100
data_augmentation = True
num_predictions = 20
#save_dir = os.path.join(os.getcwd(), 'saved_models')
#model_name = 'keras_cifar10_trained_model.h5'

In [4]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print ("y_train:", y_train.shape, y_train[0:10])
print ("y_test:", y_test.shape)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train: (50000, 1) [[6]
 [9]
 [9]
 [4]
 [1]
 [1]
 [2]
 [7]
 [8]
 [3]]
y_test: (10000, 1)


In [5]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print ("y_train:", y_train.shape)
print ("y_test:", y_test.shape)

y_train: (50000, 10)
y_test: (10000, 10)


In [14]:
x_train.shape[1:]

(32, 32, 3)

In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [7]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [8]:
print ("# of uniques in x_train:", len(np.unique(x_train)))

# of uniques in x_train: 256


In [9]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [10]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

Using real-time data augmentation.


In [11]:
# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

In [12]:
# Fit the model on the batches generated by datagen.flow().
history1 = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=epochs,
                    steps_per_epoch=len(x_train) / batch_size,
                    validation_data=(x_test, y_test),
                    workers=4)

Epoch 1/100
1563/1562 [==============================] - 79s - loss: 1.8591 - acc: 0.3133 - val_loss: 1.6156 - val_acc: 0.4136
Epoch 2/100
1563/1562 [==============================] - 78s - loss: 1.5800 - acc: 0.4188 - val_loss: 1.4487 - val_acc: 0.4742
Epoch 3/100
1563/1562 [==============================] - 78s - loss: 1.4617 - acc: 0.4685 - val_loss: 1.3119 - val_acc: 0.5424
Epoch 4/100
1563/1562 [==============================] - 80s - loss: 1.3699 - acc: 0.5071 - val_loss: 1.2286 - val_acc: 0.5683
Epoch 5/100
1563/1562 [==============================] - 80s - loss: 1.2973 - acc: 0.5340 - val_loss: 1.1943 - val_acc: 0.5796
Epoch 6/100
1563/1562 [==============================] - 79s - loss: 1.2326 - acc: 0.5602 - val_loss: 1.0675 - val_acc: 0.6244
Epoch 7/100
1563/1562 [==============================] - 78s - loss: 1.1808 - acc: 0.5807 - val_loss: 1.0207 - val_acc: 0.6383
Epoch 8/100
1563/1562 [==============================] - 78s - loss: 1.1388 - acc: 0.5988 - val_loss: 1.0084 - 

1563/1562 [==============================] - 78s - loss: 0.7674 - acc: 0.7442 - val_loss: 0.6505 - val_acc: 0.7864
Epoch 66/100
1563/1562 [==============================] - 77s - loss: 0.7624 - acc: 0.7461 - val_loss: 0.6280 - val_acc: 0.7933
Epoch 67/100
1563/1562 [==============================] - 78s - loss: 0.7625 - acc: 0.7453 - val_loss: 0.6577 - val_acc: 0.7935
Epoch 68/100
1563/1562 [==============================] - 79s - loss: 0.7651 - acc: 0.7467 - val_loss: 0.6649 - val_acc: 0.7808
Epoch 69/100
1563/1562 [==============================] - 77s - loss: 0.7653 - acc: 0.7467 - val_loss: 0.6560 - val_acc: 0.7862
Epoch 70/100
1563/1562 [==============================] - 76s - loss: 0.7610 - acc: 0.7465 - val_loss: 0.6829 - val_acc: 0.7895
Epoch 71/100
1563/1562 [==============================] - 79s - loss: 0.7666 - acc: 0.7459 - val_loss: 0.6542 - val_acc: 0.7841
Epoch 72/100
1563/1562 [==============================] - 79s - loss: 0.7640 - acc: 0.7439 - val_loss: 0.7667 - val_a

In [ ]:
# list all data in history
print(history1.history.keys())
# summarize history for accuracy
plt.plot(history1.history['acc'])
plt.plot(history1.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validate'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validate'], loc='upper left')
plt.show()

In [13]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s     
Test loss: 0.648336306047
Test accuracy: 0.7868


In [19]:
print(x_test.shape)
print(x_test[0].shape)
temp = x_test[0:5, :, :, :]
print(temp.shape)


(10000, 32, 32, 3)
(32, 32, 3)
(5, 32, 32, 3)


In [21]:
model.predict_classes(temp)

5/5 [==============================] - 0s


array([3, 8, 8, 8, 6])